<a href="https://colab.research.google.com/github/raeubaen/ml/blob/master/Conditional%20GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import models & layers from TF/keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# tensor manipulation library
import numpy as np
# plotting library
import matplotlib.pyplot as plt

# disabling eager (slower) execution
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

In [5]:
# dataset download
!wget http://www.roma1.infn.it/~giagu/datasets/ising2_conf.dat
!wget http://www.roma1.infn.it/~giagu/datasets/ising2_label.dat

# reading configurations and temperatures into numpy 
# (10k elements with 28x28 spin configuration)
conf, temp = np.loadtxt('ising2_conf.dat'), np.loadtxt('ising2_label.dat')

--2020-06-01 19:14:19--  http://www.roma1.infn.it/~giagu/datasets/ising2_conf.dat
Resolving www.roma1.infn.it (www.roma1.infn.it)... 141.108.26.1, 141.108.26.150
Connecting to www.roma1.infn.it (www.roma1.infn.it)|141.108.26.1|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19060775 (18M)
Saving to: ‘ising2_conf.dat.2’

ising2_conf.dat.2   100%[===================>]  18.18M  3.18MB/s    in 6.5s    

2020-06-01 19:14:26 (2.79 MB/s) - ‘ising2_conf.dat.2’ saved [19060775/19060775]

--2020-06-01 19:14:29--  http://www.roma1.infn.it/~giagu/datasets/ising2_label.dat
Resolving www.roma1.infn.it (www.roma1.infn.it)... 141.108.26.150, 141.108.26.1
Connecting to www.roma1.infn.it (www.roma1.infn.it)|141.108.26.150|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81025 (79K)
Saving to: ‘ising2_label.dat.1’

ising2_label.dat.1  100%[===================>]  79.13K   142KB/s    in 0.6s    

2020-06-01 19:14:31 (142 KB/s) - ‘ising2_label.dat.1’ saved [8

In [0]:
BATCH_SIZE = 512
IMG_SHAPE = (28, 28, 1)

In [0]:
# reshaping configurations into matrix form and specifying dtype
conf = conf.reshape(-1, *IMG_SHAPE).astype('float32')

temp = temp.astype('float32')

# converting data (conf and temperatures) into tensorflow Dataset
dataset = tf.data.Dataset.from_tensor_slices((conf, temp))
# shuffling and prefetching
dataset = dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE).prefetch(32)

In [0]:
def get_discriminator_model():


    img_input = layers.Input(shape=IMG_SHAPE)

    x = layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=IMG_SHAPE)(img_input)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same')(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(1)(x)

    model = keras.models.Model(img_input, x, name="discriminator")
    return model

d_model_ord = get_discriminator_model()